In [13]:
import pandas as pd
from sdatta_learn.loader.load_from_postgres import get_all_table, get_sales_between_dates
import os

In [14]:
config_dict = {'pg_port': os.environ.get('PT_PG_PORT', '5432'),
               'pg_user': os.environ.get('PT_PG_USER', 'datatiger'),
               'pg_password': os.environ.get('PT_PG_PASSWORD', 'Hwhiupwj6SZ4Sq'),
               'pg_host': os.environ.get('PT_PG_HOST', 'sdatta-pg.postgres.database.azure.com'),
               'pg_database': os.environ.get('PT_PG_DATABASE', 'postgres'),
               'sku_store_col': 'sku, store',
               'item_store_col': 'item, store',
               'date_col': 'date',
               'sales_col': 'sales',
               'avg_factor': 7,
               'minimum_sparse_val': 0.3,
               'sales_table_name': os.environ.get('PT_SALES_TABLE_NAME', 'public.f_sales_v'),
               'artikelstamm_table_name': 'l_artikelstamm', }

In [15]:
l_artikelstamm = get_all_table(table_name='l_artikelstamm',
                               pg_host=config_dict['pg_host'],
                               pg_port=config_dict['pg_port'],
                               pg_user=config_dict['pg_user'],
                               pg_password=config_dict['pg_password'],
                               pg_database='dev')

OperationalError: connection to server at "sdatta-pg.postgres.database.azure.com" (4.227.249.9), port 5432 failed: FATAL:  database "dev" does not exist


In [ ]:
f_sales_v = get_sales_between_dates(table_name='f_sales_v',
                          pg_host=config_dict['pg_host'],
                          pg_port=config_dict['pg_port'],
                          pg_user=config_dict['pg_user'],
                          pg_password=config_dict['pg_password'],
                          pg_database='dev')